In [1]:
import torch
torch.cuda.is_available()

True

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
X_train = torch.FloatTensor([0., 1., 2.])
X_train.is_cuda

False

In [6]:
X_train = X_train.to(device)
X_train.is_cuda

True

In [8]:
X_train= torch.FloatTensor([0., 1., 2.])
X_train = X_train.cuda()
X_train.is_cuda

True

In [13]:
X_train= torch.FloatTensor([0., 1., 2.])
X_train = X_train.cpu()
X_train.is_cuda

False

In [14]:
X_train= torch.FloatTensor([0., 1., 2.])
X_train.is_cuda

False

In [12]:
torch.cuda.current_device()

0

In [10]:
torch.cuda.device_count()

1

In [11]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1650'

## CUDA 성능테스트

### GPU(CUDA)

In [18]:
import torch
import torchvision.models as models
import time

# CUDA 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True

# 모델 생성 및 GPU 이동
model = models.resnet18(pretrained=True).to(device)
model.eval()

# 테스트용 입력 데이터 생성
input_tensor = torch.rand(1, 3, 224, 224).to(device)

# CUDA 이벤트 생성
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

# 워밍업 (첫 실행의 초기화 시간 제외)
for _ in range(10):
    _ = model(input_tensor)

# 성능 측정
times = []
for _ in range(100):
    start.record()
    output = model(input_tensor)
    end.record()
    
    torch.cuda.synchronize()
    times.append(start.elapsed_time(end))

# 결과 출력
avg_time = sum(times) / len(times)
print(f'평균 추론 시간: {avg_time:.2f} ms')
print(f'초당 처리량: {1000/avg_time:.2f} FPS')


평균 추론 시간: 4.10 ms
초당 처리량: 243.78 FPS


### CPU

In [19]:
import torch
import torchvision.models as models
import time

# CPU 설정
device = torch.device('cpu')

# 모델 생성
model = models.resnet18(pretrained=True).to(device)
model.eval()

# 테스트용 입력 데이터 생성
input_tensor = torch.rand(1, 3, 224, 224).to(device)

# 워밍업 (첫 실행의 초기화 시간 제외)
for _ in range(10):
    _ = model(input_tensor)

# 성능 측정
times = []
for _ in range(100):
    start_time = time.time()
    output = model(input_tensor)
    end_time = time.time()
    times.append((end_time - start_time) * 1000)  # ms 단위로 변환

# 결과 출력
avg_time = sum(times) / len(times)
print(f'평균 추론 시간: {avg_time:.2f} ms')
print(f'초당 처리량: {1000/avg_time:.2f} FPS')


평균 추론 시간: 38.27 ms
초당 처리량: 26.13 FPS
